In [1]:
import re
import pandas as pd
import numpy as np
import string
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window

#cmovpf_path = '/group/axa_malaysia/data/adm_cmovpf'
#ctrnpf_path = '/group/axa_malaysia/data/adm_ctrnpf'
#ltrnpf_path = '/group/axa_malaysia/data/adm_ltrnpf'
#clampf_path = '/group/axa_malaysia/data/adm_clampf'
#clxdpf_path = '/group/axa_malaysia/data/adm_clxdpf'
#transv_cl_quanti_psea_path = 'data/sas_403/transv_cl_quanti_psea.parquet'
#adm_mapping_path = 'ADM Mapping.xlsm'

#cal_yr = datetime.now().year
#cal_mth = datetime.now().month
#cal_day = datetime.now().day
#cal_yrm = cal_yr*100 + cal_mth
#cal_ymd = cal_yrm*100 + cal_day

def convert_dateb(x):
    x_str = str(x).strip()
    if len(x_str) == 6 and int(x_str[0]) > 7:
        return 19000000 + x
    else: 
        return 20000000 + x
_convert_dateb_udf = udf(convert_dateb, IntegerType())

errdate = to_date(lit('2999-12-31'))
def format_date(strdate):
    try: 
        return datetime.strptime(str(strdate),'%Y%m%d').strftime('%Y-%m-%d')
    except: 
        return '2999-12-31'
_format_date = udf(format_date,StringType())
    
def claims_quanti_psea(cmovpf_path,ctrnpf_path,ltrnpf_path, output_folder = 'data/sas_403/'):    
    cmovpf = spark.read.parquet(cmovpf_path)
    ctrnpf = spark.read.parquet(ctrnpf_path)
    ltrnpf = spark.read.parquet(ltrnpf_path)

    #*----------------------------------------------------------*/
    #*                          PART I                          */
    #*----------------------------------------------------------*/


    cmovpf = cmovpf[['clmpfx','clmcoy','claim','tranno','batcactyr','batcactmn',
            'transaction_date','validflag','chgbal','paymnt','chgbal_ri','paymnt_ri',
            'clstat','paycde','clrate','reqnno','receipt','batctrcde']]\
    .filter((col('clmpfx')=='CL') & (col('clmcoy')==1) & (col('validflag')==1))

    cmovpf = cmovpf.withColumn("d_tran", to_date(_format_date(_convert_dateb_udf(col('transaction_date')))))\

    cmovpf = cmovpf.withColumn("d_clo",when(col('clstat')==2, col('d_tran')).otherwise(errdate))
    cmovpf = cmovpf.withColumn("yrm", col('batcactyr')*100+col('batcactmn'))
    cmovpf = cmovpf.withColumn("paymntid",when(col('reqnno').isNotNull() & (trim(col('reqnno'))!=''),col('reqnno')).\
                               when(col('receipt').isNotNull() & (col('receipt')!=''),col('receipt')).otherwise(lit("NA")))

    cmovpf = cmovpf.withColumnRenamed("paymnt","gpaytot").\
             withColumnRenamed("chgbal","gmovtot").\
             withColumnRenamed("paymnt_ri","rpaytot").\
             withColumnRenamed("chgbal_ri","rmovtot")

    cmovpf = cmovpf.drop('transaction_date','batcactyr','batcactmn','clmpfx','clmcoy','validflag','reqnno','receipt')
    cmovpf.cache()

    # ************************* FIRST GET BASE ************************* 
    base = cmovpf.drop('gpaytot','gmovtot','rpaytot','rmovtot')
    base = base.orderBy('claim','tranno').dropDuplicates(['claim','tranno'])
    base.cache()

    # ********************** THEN AGGREGATE CMOVPF ********************* */ 
    cmovpf_sum = cmovpf.groupBy('claim','tranno').sum('gpaytot','gmovtot','rpaytot','rmovtot')\
    .withColumnRenamed('sum(gpaytot)','gpaytot')\
    .withColumnRenamed('sum(gmovtot)','gmovtot')\
    .withColumnRenamed('sum(rpaytot)','rpaytot')\
    .withColumnRenamed('sum(rmovtot)','rmovtot')

    #*----------------------------------------------------------*/
    #*                          PART II                         */
    #*----------------------------------------------------------*/

    # Get detail of Gross Claim info in CTRNPF

    ctrnpf = ctrnpf[['clmpfx','clmcoy','claim','tranno','prcl','rscd','paymnt','chgbal']]\
             .filter((col('clmpfx')=='CL') & (col('clmcoy')==1))

    ctrnpf = ctrnpf.withColumn("prcl_rscd", concat(col('prcl'), col('rscd')))
    ctrnpf = ctrnpf.withColumnRenamed("paymnt","gpay").withColumnRenamed("chgbal","gmov")
    ctrnpf.cache()

    classes = ctrnpf[['claim','prcl_rscd']].orderBy(['claim','prcl_rscd']).dropDuplicates(['claim','prcl_rscd'])
    trannos = cmovpf[['claim','tranno']].orderBy(['claim','tranno']).dropDuplicates(['claim','tranno'])

    template = trannos.join(classes, trannos['claim'] == classes['claim'], 'left')
    template = template.select([trannos[c] for c in trannos.columns]+[classes['prcl_rscd']])
    template = template.orderBy('claim', 'tranno', 'prcl_rscd')

    # Remove records which does not exist in CMOVPF 
    # since we need to recon with CMOVPF
    ctrnpf = base[['claim','tranno']].join(ctrnpf, ['claim','tranno'], 'left').fillna(0, subset=['gpay','gmov'])

    # Aggregate on granular level
    ctrnpf = ctrnpf.groupBy('claim','tranno','prcl_rscd').sum('gpay','gmov')\
    .withColumnRenamed('sum(gpay)','gpay')\
    .withColumnRenamed('sum(gmov)','gmov')

    ctrnpf = template.join(ctrnpf,on=['claim','tranno','prcl_rscd'],how='left').fillna(0, subset=['gpay','gmov'])

    # Aggregate gross claim info in CTRNPF for recon
    ctrnpf_sum = ctrnpf.groupBy('claim','tranno').agg(sum('gpay'),sum('gmov'),count(lit(1)))\
    .withColumnRenamed('sum(gpay)','gpay')\
    .withColumnRenamed('sum(gmov)','gmov')\
    .withColumnRenamed('count(1)','nbkeys')
    ctrnpf_sum.cache()

    #*-----------------------------------------------------------*/
    #*                          PART III                         */
    #*-----------------------------------------------------------*/

    recon_cl_gr_psea_recon = cmovpf_sum.drop('rpaytot','rmovtot').join(ctrnpf_sum.drop('nbkeys'),on=['claim','tranno'],how='left')\
    .fillna(0, subset=['gpay','gmov'])\
    .withColumn('err_pay',when(abs(col('gpay')-col('gpaytot'))>0.01,lit(1)).otherwise(lit(0)))\
    .withColumn('err_mov',when(abs(col('gmov')-col('gmovtot'))>0.01,lit(1)).otherwise(lit(0)))\
    .filter((col('err_pay')==1) | (col('err_mov')==1))

    recon_cl_gr_psea_recon.write.parquet('{}recon_cl_gr_psea_recon.parquet'.format(output_folder))

    #*-----------------------------------------------------------*/
    #*                          PART IV                          */
    #*-----------------------------------------------------------*/
    # Aggregate reinsurance claim info in LTRNPF for recon
    ltrnpf = ltrnpf[['clmpfx','clmcoy','claim','tranno','lactyp','paymnt_ri','chgbal_ri']]\
    .filter((col('clmpfx')=='CL') & (col('clmcoy')==1))\
    .withColumn('cpay',when(col('lactyp')=='I',col('paymnt_ri')).otherwise(lit(0)))\
    .withColumn('cmov',when(col('lactyp')=='I',col('chgbal_ri')).otherwise(lit(0)))\
    .withColumn('rpay',when(col('lactyp')=='I',lit(0)).otherwise(col('paymnt_ri')))\
    .withColumn('rmov',when(col('lactyp')=='I',lit(0)).otherwise(col('chgbal_ri')))\
    .drop('clmpfx','clmcoy','lactyp','paymnt_ri','chgbal_ri')

    # Remove records which don't exist in CMOVPF since we need to recon with CMOVPF

    ltrnpf = base[['claim','tranno']].join(ltrnpf, on=['claim','tranno'],how='left').fillna(0, subset=['cpay','cmov','rpay','rmov'])
    ltrnpf.cache()
    # Aggregate on granular level - here it is claim tranno since this is the most granular level
    # (and we need it more granular we will merge it later)

    ltrnpf_sum = ltrnpf.groupBy('claim','tranno').sum('cpay','cmov','rpay','rmov')\
    .withColumnRenamed('sum(cpay)','cpay').withColumnRenamed('sum(cmov)','cmov')\
    .withColumnRenamed('sum(rpay)','rpay').withColumnRenamed('sum(rmov)','rmov')

    #*----------------------------------------------------------*/
    #*                          PART V                          */
    #*----------------------------------------------------------*/
    # Reconcile the data
    # Recon - can only recon reinsurance total and not split by RI and CO, since in the 
    # CMOVPF table we do not have the information on the reinsurance
    # What this means is that we don't have an rpaytot and a cpaytot in CMOVPF_SUM,
    # we are unable to split in CMOVPF between CO in RI.

    recon_cl_ri_psea_recon = cmovpf_sum.drop('gmovtot','gpaytot').join(ltrnpf_sum,on=['claim','tranno'],how='left')\
    .fillna(0, subset=['cpay','cmov','rpay','rmov'])\
    .withColumn('err_pay',when(abs(col('cpay')+col('rpay')-col('rpaytot'))>0.01,lit(1)).otherwise(lit(0)))\
    .withColumn('err_mov',when(abs(col('cmov')+col('rmov')-col('rmovtot'))>0.01,lit(1)).otherwise(lit(0)))\
    .filter((col('err_pay')==1) | (col('err_mov')==1))

    recon_cl_ri_psea_recon.write.parquet('{}recon_cl_ri_psea_recon.parquet'.format(output_folder))

    #*-----------------------------------------------------------*/
    #*                          PART VI                          */
    #*-----------------------------------------------------------*/
    # Aggregate LTRNPF
    # Note on retain - it
    ltrnpf_sum2 = ltrnpf_sum\
    .withColumn('cpay_cum_tranno',sum('cpay').over(Window.partitionBy('claim').orderBy('tranno')))\
    .withColumn('cmov_cum_tranno',sum('cmov').over(Window.partitionBy('claim').orderBy('tranno')))\
    .withColumn('rpay_cum_tranno',sum('rpay').over(Window.partitionBy('claim').orderBy('tranno')))\
    .withColumn('rmov_cum_tranno',sum('rmov').over(Window.partitionBy('claim').orderBy('tranno')))\
    .withColumnRenamed('cpay','cpay_aggr')\
    .withColumnRenamed('cmov','cmov_aggr')\
    .withColumnRenamed('rpay','rpay_aggr')\
    .withColumnRenamed('rmov','rmov_aggr')

    # Aggregate CTRNPF
    ctrnpf_sum2 = ctrnpf_sum\
    .withColumn('gpay_cum_tranno',sum('gpay').over(Window.partitionBy('claim').orderBy('tranno')))\
    .withColumn('gmov_cum_tranno',sum('gmov').over(Window.partitionBy('claim').orderBy('tranno')))\
    .withColumnRenamed('gpay','gpay_aggr')\
    .withColumnRenamed('gmov','gmov_aggr')

    # Merge
    # Use this instead of making a condition on each Numeric field that if it is . you make it 0
    quanti_psea = ctrnpf.join(ltrnpf_sum2, on=['claim','tranno'],how ='outer')\
                        .join(ctrnpf_sum2, on =['claim','tranno'],how ='left').fillna(0)


    # Proportion the reinsurance to granularity
    # Cumulate on gpay and gmov since this is all you know up to now
    quanti_psea2 = quanti_psea\
    .withColumn('gpay_cum_pr_rsv',sum('gpay').over(Window.partitionBy('claim','prcl_rscd').orderBy('prcl_rscd','tranno')))\
    .withColumn('gmov_cum_pr_rsv',sum('gmov').over(Window.partitionBy('claim','prcl_rscd').orderBy('prcl_rscd','tranno')))

    quanti_psea2 = quanti_psea2\
    .withColumn('prop_pay', when(col('gpay_cum_tranno')!=0, col('gpay_cum_pr_rsv')/col('gpay_cum_tranno')).\
                when(col('nbkeys')!=0, 1/col('nbkeys')).otherwise(lit(1)))\
    .withColumn('prop_mov', when(col('gmov_cum_tranno')!=0, col('gmov_cum_pr_rsv')/col('gmov_cum_tranno')).\
               when(col('nbkeys')!=0, 1/col('nbkeys')).otherwise(lit(1)))\
    .withColumn('cpay_cum_pr_rsv',col('prop_pay') * col('cpay_cum_tranno'))\
    .withColumn('rpay_cum_pr_rsv',col('prop_pay') * col('rpay_cum_tranno'))\
    .withColumn('cmov_cum_pr_rsv',col('prop_mov') * col('cmov_cum_tranno'))\
    .withColumn('rmov_cum_pr_rsv',col('prop_mov') * col('rmov_cum_tranno'))

    # Get incremental for CO and RI
    quanti_psea2 = quanti_psea2\
    .withColumn('firstclaim',row_number().over(Window.partitionBy('claim').orderBy('prcl_rscd','tranno')))\
    .withColumn('firstprcl',row_number().over(Window.partitionBy('claim','prcl_rscd').orderBy('tranno')))\
    .withColumn('lastclaim',row_number().over(Window.partitionBy('claim').orderBy(desc('prcl_rscd'),desc('tranno'))))\
    .withColumn('lastprcl',row_number().over(Window.partitionBy('claim','prcl_rscd').orderBy(desc('tranno'))))\
    .withColumn('cpay',when(col('firstprcl')==1,col('cpay_cum_pr_rsv'))\
               .otherwise(col('cpay_cum_pr_rsv')-lag(col('cpay_cum_pr_rsv')).over(Window.partitionBy('claim','prcl_rscd').orderBy('prcl_rscd','tranno'))))\
    .withColumn('rpay',when(col('firstprcl')==1,col('rpay_cum_pr_rsv'))\
               .otherwise(col('rpay_cum_pr_rsv')-lag(col('rpay_cum_pr_rsv')).over(Window.partitionBy('claim','prcl_rscd').orderBy('prcl_rscd','tranno'))))\
    .withColumn('cmov',when(col('firstprcl')==1,col('cmov_cum_pr_rsv'))\
               .otherwise(col('cmov_cum_pr_rsv')-lag(col('cmov_cum_pr_rsv')).over(Window.partitionBy('claim','prcl_rscd').orderBy('prcl_rscd','tranno'))))\
    .withColumn('rmov',when(col('firstprcl')==1,col('rmov_cum_pr_rsv'))\
               .otherwise(col('rmov_cum_pr_rsv')-lag(col('rmov_cum_pr_rsv')).over(Window.partitionBy('claim','prcl_rscd').orderBy('prcl_rscd','tranno'))))\
    .withColumn('rmov_cmov_error_flag',when((col('lastclaim')==1) |
                                            (col('lastprcl')==1) |
                                            ((col('gmov_cum_pr_rsv')==0) & (col('rmov_cum_pr_rsv')!=0)), lit(1)))\
    .select('claim','tranno','prcl_rscd','gpay','gmov','cpay','cmov','rpay','rmov')

    # 	/*----------------------------------------------------------*/
    # 	/* 						  PART VII							*/
    # 	/*----------------------------------------------------------*/

    # Finalize by adding qualitative information
    cl_quanti_psea = base.join(quanti_psea2,on=['claim','tranno'], how='left')

    #/* Manual Adjustments from Mapping File :
    #		Cases where Balo of CLAMPF doesn't reconcile
    #		with the sum of CHGBAL in CMOVPF			*/
    #MISSING

    cl_quanti_psea.write.parquet('{}transv_cl_quanti_psea.parquet'.format(output_folder))

In [5]:
def claims_quali_psea(transv_cl_quanti_psea_path, clampf_path, clxdpf_path, cal_ymd, adm_mapping_path, output_folder = 'data/sas_403/'):
    
    transv_cl_quanti_psea = spark.read.parquet(transv_cl_quanti_psea_path)
    clampf = spark.read.parquet(clampf_path)
    clxdpf = spark.read.parquet(clxdpf_path)
    
    #******************************************************************
    #**************************** GET BASE **************************** 
    #******************************************************************
    
    basequali = transv_cl_quanti_psea[['claim','d_tran','d_clo']]
    basequali.cache()

    # Keep firsts registration date only
    basequalireg = basequali.drop('d_clo').withColumnRenamed('d_tran','d_reg').orderBy('claim').dropDuplicates(['claim'])

    # Keep closing date only
    basequaliclo = basequali.withColumn('rownum',row_number().over(Window.partitionBy('claim').orderBy(desc('D_tran'))))
    basequaliclo = basequaliclo.filter(col('rownum')==1).drop('d_tran','rownum')

    basequali1 = basequalireg.join(basequaliclo,on='claim',how='inner')

    #******************************************************************
    #******************* CLEAN QUALITATIVE TABLES *********************
    #******************************************************************
    
    clampf = clampf.select('claim','chdrstcda','chdrstcdc','agntnum','clntnum','chdrnum',
                           'datrep','datocc','rskno','cnttype','rsktyp','clmdsc',
                           'cedref','zrepclmno','assess','assessdt','solict','solictdt',
                           'clrvwdat','user_profile','clstat','clcurr','subrec','id',
                           'mevent','validflag','coppn','servbr','tparty')\
    .filter(col('validflag')==1).orderBy('claim').dropDuplicates(subset=['claim'])

    # CLXDPF - Extra claim info
    # Normally each row should be unique by claim
    
    clxdpf = clxdpf.select('claim','ccdate','dteeff','crdate','acstyp','desc01','desc02','validflag')\
    .filter(col('validflag')==1).orderBy('claim').dropDuplicates(subset=['claim'])

    # Merge with base and clean variables
    cl_quali_psea = basequali1.join(clampf,on='claim',how='left').join(clxdpf, on='claim',how='left')
    
    # Format dates
    # Reported, occurred appointment and review dates
    
    cl_quali_psea = cl_quali_psea\
    .withColumn('d_com', to_date(_format_date(col('ccdate'))))\
    .withColumn('d_exp', to_date(_format_date(col('crdate'))))\
    .withColumn('d_eff', to_date(_format_date(col('dteeff'))))\
    .withColumn('d_rep', to_date(_format_date(col('datrep'))))\
    .withColumn('d_occ', to_date(_format_date(col('datocc'))))\
    .withColumn('assessdt', when(col('assessdt')>cal_ymd,lit(0)).otherwise(col('assessdt')))\
    .withColumn('d_app_a', to_date(_format_date(col('assessdt'))))\
    .withColumn('d_app_s', to_date(_format_date(col('solictdt'))))\
    .withColumn('d_review', to_date(_format_date(col('clrvwdat'))))\
    .withColumnRenamed('chdrstcda','fundcode')\
    .withColumnRenamed('agntnum','agentid')\
    .withColumnRenamed('clntnum','claimantid')\
    .withColumn('clmdesc1',lower(trim(col('clmdsc'))))\
    .withColumn('clmdesc2',lower(trim(col('desc01'))))\
    .withColumn('clmdesc3',lower(trim(col('desc02'))))\
    .withColumnRenamed('user_profile','examiner')\
    .withColumnRenamed('id','officerid')\
    .withColumn('assessor',when(col('assess').isNotNull(), col('assess')).otherwise(lit('NA')))\
    .withColumn('solicitor',when(col('solict').isNotNull(), col('solict')).otherwise(lit('NA')))\
    .drop('datrep','datocc','clmdsc','validflag',
          'clrvwdat','assess','solict','soilctdt','assessdt',
          'ccdate','dteeff','crdate','desc01','desc02')
    
    # Read in the mappings file
    
    # Nat Cat
    natcat_map = pd.read_excel(adm_mapping_path, sheetname = '111')
    natcat_map['mevent'] = natcat_map['mevent'].astype(str)
    
    transv_cl_quali_psea = cl_quali_psea.join(broadcast(spark.createDataFrame(natcat_map)),on=['chdrstcdc','mevent'],how='left')\
    .drop('clstat').fillna(0,'natcat')
    
    transv_cl_quali_psea.write.parquet('{}transv_cl_quali_psea.parquet'.format(output_folder))

In [2]:
cmovpf_path = '/group/axa_malaysia/data/adm_cmovpf'
ctrnpf_path = '/group/axa_malaysia/data/adm_ctrnpf'
ltrnpf_path = '/group/axa_malaysia/data/adm_ltrnpf'
clampf_path = '/group/axa_malaysia/data/adm_clampf'
clxdpf_path = '/group/axa_malaysia/data/adm_clxdpf'
transv_cl_quanti_psea_path = 'data/sas_403/transv_cl_quanti_psea.parquet'
adm_mapping_path = 'ADM Mapping.xlsm'
cal_yr = datetime.now().year
cal_mth = datetime.now().month
cal_day = datetime.now().day
cal_yrm = cal_yr*100 + cal_mth
cal_ymd = cal_yrm*100 + cal_day


In [3]:
claims_quanti_psea(cmovpf_path, ctrnpf_path, ltrnpf_path)

In [6]:
claims_quali_psea(transv_cl_quanti_psea_path, clampf_path, clxdpf_path, cal_ymd, adm_mapping_path)

In [44]:
cl_quanti_psea.count()

4939745

In [45]:
cl_quanti_psea.groupby().sum('gpay','gmov','cpay','cmov','rpay','rmov').collect()

[Row(sum(gpay)=4038300006.249997, sum(gmov)=721206971.5299996, sum(cpay)=111049582.85000004, sum(cmov)=30006078.270000014, sum(rpay)=642499920.3862486, sum(rmov)=144475273.68999955)]

In [14]:
testing = spark.createDataFrame([[300,1,1],[250,2,2],[100,3,3],[75,4,4]],['rmov_cum_pr_rsv','firstclaim','firstprcl'])

In [15]:
testing.show()

+---------------+----------+---------+
|rmov_cum_pr_rsv|firstclaim|firstprcl|
+---------------+----------+---------+
|            300|         1|        1|
|            250|         2|        2|
|            100|         3|        3|
|             75|         4|        4|
+---------------+----------+---------+



In [21]:
testing = testing.withColumn('tmp_rmov_cum_pr_rsv', when((col('firstclaim')==1)|(col('firstprcl')==1), col('rmov_cum_pr_rsv'))\
           .otherwise(-col('rmov_cum_pr_rsv')))\
.withColumn('rmov', sum('tmp_rmov_cum_pr_rsv').over(Window.partitionBy().orderBy('firstclaim')))

In [22]:
testing.show()

+---------------+----------+---------+-------------------+----+
|rmov_cum_pr_rsv|firstclaim|firstprcl|tmp_rmov_cum_pr_rsv|rmov|
+---------------+----------+---------+-------------------+----+
|            300|         1|        1|                300| 300|
|            250|         2|        2|               -250|  50|
|            100|         3|        3|               -100| -50|
|             75|         4|        4|                -75|-125|
+---------------+----------+---------+-------------------+----+



In [37]:
testing = testing.withColumn('rmovtest',col('rmov_cum_pr_rsv')-lag(col('rmov_cum_pr_rsv')).over(Window.partitionBy().orderBy('firstclaim')))

In [38]:
testing.withColumn('rmovtest',when(col('firstclaim')==1,col('rmov_cum_pr_rsv')).otherwise(col('rmovtest'))).show()

+---------------+----------+---------+-------------------+----+--------+
|rmov_cum_pr_rsv|firstclaim|firstprcl|tmp_rmov_cum_pr_rsv|rmov|rmovtest|
+---------------+----------+---------+-------------------+----+--------+
|            300|         1|        1|                300| 300|     300|
|            250|         2|        2|               -250|  50|     -50|
|            100|         3|        3|               -100| -50|    -150|
|             75|         4|        4|                -75|-125|     -25|
+---------------+----------+---------+-------------------+----+--------+



In [1]:
adm_mapping_path = 'ADM Mapping.xlsm'
ztrnpf_path = '/river/axa_my/axa_aaro_psea/data/psea_ztrnpf/merge/yyyy=2027/mm=10/dd=31'
monthend = 1
import re
import pandas as pd
import numpy as np
import string
from datetime import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window
def format_date(strdate):
    try: 
        return datetime.strptime(str(strdate),'%Y%m%d').strftime('%Y-%m-%d')
    except: 
        return '2999-12-31'
_format_date = udf(format_date,StringType())

"""
    /* 1) sacscode filter */
    /* Consider Gross Prem (FG) and Co-insurance Prem      */
    /* we will consider RP (Re-insurance premium) in  	   */
    /* the next section, since it only needs to run   	   */
    /* once a month - it is a massive table			       */
    /* 2) batctrcde filter */
    /* Only consider real financial transactions      	   */
"""

"""
    /* Map Transaction Types */
    /* Note - the mapping is limited to the filter, so the mapping is only needed for the
       transaction types applied in the filter */
    Transaction Types are read from the excel mapping files
"""
transtype_map = pd.read_excel(adm_mapping_path, sheetname='300').set_index('batctrcde').to_dict()['trantype']
transtype_map_bc = sc.broadcast(transtype_map)

ztrnpf = spark.read.parquet(ztrnpf_path)[['batcactyr',
 'batcactmn',
 'rldgacct',
 'tranno',
 'ccdate',
 'effdate',
 'accnum',
 'expiry_date',
 'batctrcde',
 'sacscode',
 'trandate',
 'chdrstcdc',
 'tranamt01',
 'tranamt02',
 'tranamt03',
 'tranamt04',
 'tranamt05',
 'tranamt14',
 'batcbrn']].filter( col("batctrcde").isin(['TA39','T409','T44B','T922','T405','T903','T413','T927','T928',
                                            'BA25','T454','T467','T913','T914','T926','T930','T934','T931']) )#.limit(100000)
#'B470','T46B','T475','T840','B920','BR9A','T8A0','T933'
ztrnpf_fgco = ztrnpf
"""
ztrnpf_fgco = ztrnpf.filter(col("sacscode").isin("FG","CO"))
ztrnpf_rp = ztrnpf.filter(col("sacscode")=='RP')
"""
def transformations(df):
    df = df.withColumn('yrm', col('batcactyr')*100+col('batcactmn'))

    #dates formatting
    df = df.withColumn('d_tran', to_date(_format_date(col('trandate'))))\
    .withColumn('d_eff', to_date(_format_date(col('effdate'))))\
    .withColumn('d_com', to_date(_format_date(col('ccdate'))))\
    .withColumn('d_exp', to_date(_format_date(col('expiry_date'))))

    df = df.drop('batcactyr','batcactmn','effdate','ccdate','expiry_date','trandate','tranamt01','tranamt02','tranamt03','tranamt04',
                 'tranamt05','tranamt14')

    #map transaction types
    df = df.withColumn('trantype', udf(lambda x: transtype_map_bc.value.get(x,'NA'))(col('batctrcde')))
    return df

df1 = ztrnpf.withColumnRenamed('rldgacct', 'chdrnum')\
            .withColumnRenamed('accnum', 'agentid')

df2 = df1.withColumn('fg_gr_premium_total', when(col('sacscode').isin(['FG','GR']), col('tranamt01') - col('tranamt03') + col('tranamt14')).otherwise(0) )\
.withColumn('fg_gr_commission_total', when(col('sacscode').isin(['FG','GR']), col('tranamt04') + col('tranamt05')).otherwise(0) )\
.withColumn('co_premium_total', when(col('sacscode')=='CO', col('tranamt01') - col('tranamt03') + col('tranamt14')).otherwise(0) )\
.withColumn('co_commission_total', when(col('sacscode')=='CO', col('tranamt04') + col('tranamt05')).otherwise(0) )\
.withColumn('rp_premium_total', when(col('sacscode')=='RP', col('tranamt01') - col('tranamt03') + col('tranamt14')).otherwise(0) )\
.withColumn('rp_commission_total', when(col('sacscode')=='RP', col('tranamt04') + col('tranamt05')).otherwise(0) )

df3 = transformations(df2)

df4 = df3.groupBy('chdrnum','tranno', 'batcbrn', 'yrm').sum('fg_gr_premium_total',
                                          'fg_gr_commission_total',
                                          'co_premium_total',
                                          'co_commission_total',
                                          'rp_premium_total',
                                          'rp_commission_total')\
.withColumnRenamed('sum(fg_gr_premium_total)','fg_gr_premium_total')\
.withColumnRenamed('sum(fg_gr_commission_total)','fg_gr_commission_total')\
.withColumnRenamed('sum(co_premium_total)','co_premium_total')\
.withColumnRenamed('sum(co_commission_total)','co_commission_total')\
.withColumnRenamed('sum(rp_premium_total)','rp_premium_total')\
.withColumnRenamed('sum(rp_commission_total)','rp_commission_total')

In [4]:
ztrnpf.filter(col('rldgacct')=='01748131').show()

+---------+---------+--------+------+--------+--------+------+-----------+---------+--------+--------+---------+---------+---------+---------+---------+---------+---------+-------+
|batcactyr|batcactmn|rldgacct|tranno|  ccdate| effdate|accnum|expiry_date|batctrcde|sacscode|trandate|chdrstcdc|tranamt01|tranamt02|tranamt03|tranamt04|tranamt05|tranamt14|batcbrn|
+---------+---------+--------+------+--------+--------+------+-----------+---------+--------+--------+---------+---------+---------+---------+---------+---------+---------+-------+
|     2008|        4|01748131|     2|20080328|20080328| 22929|   20090327|     T928|      GR|20080402|      AHX|  1727.00|    10.00|     0.00|   259.05|     0.00|     0.00|     89|
|     2017|        5|01748131|    11|20170328|20170328| 22929|   20180327|     T928|      GR|20170511|      AHX|  4289.00|    10.00|     0.00|   643.35|     0.00|     0.00|     89|
|     2014|        4|01748131|     8|20140328|20140328| 22929|   20150327|     T928|      GR|20

In [2]:
df3 = df3[['chdrnum','tranno', 'batcbrn', 'yrm', 'sacscode', 'batctrcde',
     'fg_gr_premium_total',
     'fg_gr_commission_total',
     'co_premium_total',
     'co_commission_total',
     'rp_premium_total',
     'rp_commission_total'
    ]].withColumn('tranno',col('tranno').cast('string'))\
.withColumn('yrm',col('yrm').cast('string'))\
.withColumn('fg_gr_premium_total',col('fg_gr_premium_total').cast('float'))\
.withColumn('fg_gr_commission_total',col('fg_gr_commission_total').cast('float'))\
.withColumn('co_premium_total',col('co_premium_total').cast('float'))\
.withColumn('co_commission_total',col('co_commission_total').cast('float'))\
.withColumn('rp_premium_total',col('rp_premium_total').cast('float'))\
.withColumn('rp_commission_total',col('rp_commission_total').cast('float'))

In [6]:
df3.filter(col('chdrnum')=='01748131').show()

+--------+------+-------+------+--------+---------+----------------+-------------------+----------------+-------------------+----------------+-------------------+
| chdrnum|tranno|batcbrn|   yrm|sacscode|batctrcde|fg_premium_total|fg_commission_total|co_premium_total|co_commission_total|rp_premium_total|rp_commission_total|
+--------+------+-------+------+--------+---------+----------------+-------------------+----------------+-------------------+----------------+-------------------+
|01748131|     2|     89|200804|      GR|     T928|             0.0|                0.0|             0.0|                0.0|             0.0|                0.0|
|01748131|    11|     89|201705|      GR|     T928|             0.0|                0.0|             0.0|                0.0|             0.0|                0.0|
|01748131|     8|     89|201404|      GR|     T928|             0.0|                0.0|             0.0|                0.0|             0.0|                0.0|
|01748131|     7|     

In [4]:
df3.write.saveAsTable('axa_malaysia.pr_gr_psea24', mode='overwrite')

In [11]:
df4 = df4.withColumn('tranno',col('tranno').cast('string')).withColumn('yrm',col('yrm').cast('string'))

In [12]:
df4.write.saveAsTable('axa_malaysia.pr_gr_psea18', mode='overwrite')